<a href="https://colab.research.google.com/github/yskmt2018/quantum/blob/master/vehicle_routing_problem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Vehicle Routing Problem with Quantum Annealing

* This notebook was created and tested on Google Colaboratory. (2020/09)

## Reference

> 齋藤和広, 大山重樹, 梅木智光, 黒川茂莉, 小野智弘, "配送計画問題への量子アニーリング適用に関する初期評価"
>
> DEIM2020 D2-4(day1 p25) https://proceedings-of-deim.github.io/DEIM2020/papers/D2-4.pdf

In [1]:
!python -V
!pip install --quiet pyqubo openjij dwave-ocean-sdk

Python 3.6.9


In [2]:
import math
import pyqubo
import numpy as np
import pandas as pd
from google import colab
from configparser import ConfigParser
from openjij import SQASampler
from dwave.system import LeapHybridSampler
from IPython.display import display_html

## Introduction

* 配送計画問題（VRP: Vehicle Routing Problem）は、複数配送車の配送順序を最適化する組合せ最適化問題であり、巡回セールスマン問題（TSP: Traveling Salesman Problem）を一般化した問題である。

* 物流における物品配送や工場内の部品移動、デマンド交通サービスにおける配車計画など、様々な現実問題に応用可能な問題である。

* VRP は代表的な組合せ最適化問題の一つとして NP-Hard（NP困難）と呼ばれる問題であり、従来のコンピュータでは多項式時間で厳密解を求めることが困難であることが知られている。

* 量子アニーリングは、量子効果を利用して NP-Hard な組合せ最適化問題を現実時間で解くことが期待されている。

### Problem Settings

* 基礎的な VRP として、複数の配送車で対象となる全ての拠点を巡回する場合に、総移動コストが最小となる巡回ルートを探索する組合せ最適化問題を考える。

* 全ての配送車は Depot と呼ばれる拠点を出発点とし、必ず最後に Depot に帰還する問題を想定する。移動コストは拠点間の距離を利用する。

In [3]:
# Cost Matrix (Distance between locations)
distance = np.array([
  [0, 548, 776, 696, 582, 274, 502, 194, 308, 194, 536, 502, 452, 1020, 393, 208, 1196, 983, 427, 482, 479, 517],
  [548, 0, 684, 308, 194, 502, 730, 354, 696, 742, 1084, 594, 1102, 298, 741, 582, 712, 1085, 1024, 202, 220, 1050],
  [776, 684, 0, 992, 878, 502, 274, 810, 468, 742, 400, 1278, 1152, 819, 510, 514, 399, 537, 579, 509, 285, 555],
  [696, 308, 992, 0, 114, 650, 878, 502, 844, 890, 1232, 514, 300, 263, 593, 153, 801, 1128, 729, 1230, 317, 946],
  [582, 194, 878, 114, 0, 536, 764, 388, 730, 776, 1118, 400, 511, 886, 168, 514, 396, 594, 886, 190, 729, 1042],
  [274, 502, 502, 650, 536, 0, 228, 308, 194, 240, 582, 776, 270, 343, 716, 535, 1269, 410, 1177, 494, 1133, 651],
  [502, 730, 274, 878, 764, 228, 0, 536, 194, 468, 354, 1004, 144, 353, 408, 220, 636, 514, 1180, 1146, 130, 648],
  [194, 354, 810, 502, 388, 308, 536, 0, 342, 388, 730, 468, 476, 1296, 338, 1100, 1125, 986, 710, 320, 279, 460],
  [308, 696, 468, 844, 730, 194, 194, 342, 0, 274, 388, 810, 263, 138, 145, 740, 1251, 1262, 402, 569, 189, 877],
  [194, 742, 742, 890, 776, 240, 468, 388, 274, 0, 342, 536, 1190, 757, 293, 580, 1195, 124, 268, 505, 309, 602],
  [536, 1084, 400, 1232, 1118, 582, 354, 730, 388, 342, 0, 878, 369, 161, 306, 1074, 1110, 1130, 347, 163, 817, 637],
  [502, 594, 1278, 514, 400, 776, 1004, 468, 810, 536, 878, 0, 381, 1041, 1004, 1251, 406, 578, 1231, 584, 727, 902],
  [452, 1102, 1152, 300, 511, 270, 144, 476, 263, 1190, 369, 381, 0, 720, 1011, 991, 957, 233, 758, 934, 1054, 154],
  [1020, 298, 819, 263, 886, 343, 353, 1296, 138, 757, 161, 1041, 720, 0, 156, 958, 1202, 919, 401, 685, 716, 994],
  [393, 741, 510, 593, 168, 716, 408, 338, 145, 293, 306, 1004, 1011, 156, 0, 798, 270, 298, 618, 246, 766, 957],
  [208, 582, 514, 153, 514, 535, 220, 1100, 740, 580, 1074, 1251, 991, 958, 798, 0, 1244, 516, 710, 1086, 946, 1098],
  [1196, 712, 399, 801, 396, 1269, 636, 1125, 1251, 1195, 1110, 406, 957, 1202, 270, 1244, 0, 1112, 1031, 1106, 785, 913],
  [983, 1085, 537, 1128, 594, 410, 514, 986, 1262, 124, 1130, 578, 233, 919, 298, 516, 1112, 0, 593, 109, 649, 1150],
  [427, 1024, 579, 729, 886, 1177, 1180, 710, 402, 268, 347, 1231, 758, 401, 618, 710, 1031, 593, 0, 676, 785, 602],
  [482, 202, 509, 1230, 190, 494, 1146, 320, 569, 505, 163, 584, 934, 685, 246, 1086, 1106, 109, 676, 0, 394, 263],
  [479, 220, 285, 317, 729, 1133, 130, 279, 189, 309, 817, 727, 1054, 716, 766, 946, 785, 649, 785, 394, 0, 826],
  [517, 1050, 555, 946, 1042, 651, 648, 460, 877, 602, 637, 902, 154, 994, 957, 1098, 913, 1150, 602, 263, 826, 0],
])

In [4]:
def is_symmetric(a: np.ndarray) -> bool:
  return np.array_equal(a, a.T)

is_symmetric(distance)

True

In [5]:
# Vehicle Number
V = 3
# Location Number
L = len(distance[0])
# Sequence Number
S = 10

print('V:{}, L:{}, S:{}'.format(V, L, S))
print('Variable Number(Use Qubit):{}'.format(V*L*S))
print('All Patterns: {:.0f} Digits'.format(math.log10(2**(V*L*S))))

V:3, L:22, S:10
Variable Number(Use Qubit):660
All Patterns: 199 Digits


* TIPS：観測可能宇宙内の原子数は約10^80

## Formulation

* VRP の目的関数及び制約条件を定式化し、QUBO（Quadratic Unconstrained Binary Optimization）式を構築する。

* QUBO 式の構築には、OSS ライブラリ PyQUBO（[GitHub](https://github.com/recruit-communications/pyqubo), [Documentation](https://pyqubo.readthedocs.io/en/latest/)）を用いる。

### QUBO1: Objective Function

* 目的関数：総移動コスト

$$
\sum_{v \in V, s \in S, (i,j) \in L} d_{ij} * x_{vsi} * x_{v(s+1)j}
$$

In [6]:
def build_objective(x: pyqubo.array.Array) -> pyqubo.core.express.AddList:
  H = sum(distance[l_from][l_to] * x[v][s][l_from] * x[v][s+1][l_to]
          for v in range(V)
          for s in range(S-1)
          for l_from in range(L)
          for l_to in range(L)
          )
  return H

### QUBO2: Depot start rule

* 制約条件：最初の訪問では必ず Depot に滞在する。

$$
w_0\sum_{v \in V} (x_{v00} - 1)^2
$$

In [7]:
def build_depot_start_rule(x: pyqubo.array.Array) -> pyqubo.core.express.Constraint:
  H = pyqubo.Constraint(
      sum((x[v][0][0] - 1)**2 for v in range(V)),
      'w_0')
  return H

### QUBO3: Depot end rule

* 制約条件：最後の訪問では必ず Depot に滞在する。

$$
w_1\sum_{v \in V} (x_{v(S-1)0} - 1)^2
$$

In [8]:
def build_depot_end_rule(x: pyqubo.array.Array) -> pyqubo.core.express.Constraint:
  H = pyqubo.Constraint(
      sum((x[v][S-1][0] - 1)**2 for v in range(V)),
      'w_1')
  return H

### QUBO4: Vehicle stay rule

* 制約条件：全ての配送車及び全ての訪問順番で、必ず一つの拠点に訪問する。

$$
w_2\sum_{v \in V, s \in S}\left(\sum_{l \in L} x_{vsl} - 1\right)^2
$$

In [9]:
def build_vehicle_stay_rule(x: pyqubo.array.Array) -> pyqubo.core.express.Constraint:
  H = pyqubo.Constraint(
      sum(
          (sum(x[v][s][l] for l in range(L)) - 1)**2
          for v in range(V)
          for s in range(S)),
          'w_2')
  return H

### QUBO5: Location visit rule

* 制約条件：いずれかの配送車及びいずれかの訪問順番で、Depot 以外の全ての拠点が必ず一回ずつ訪問される。

$$
w_3\sum_{l \in L, l!=0}\left(\sum_{v \in V, s \in S} x_{vsl} - 1\right)^2
$$

In [10]:
def build_location_visit_rule(x: pyqubo.array.Array) -> pyqubo.core.express.Constraint:
  H = pyqubo.Constraint(
      sum(
          (sum(x[v][s][l] for v in range(V) for s in range(S)) - 1)**2
          for l in range(1, L)),
          'w_3')
  return H

* 四つの制約条件を全て満たしたうえで、最小コストとなる各配送車の訪問ルートを算出する。

### Hamiltonian

* 目的関数及び制約条件からハミルトニアン H を以下のように定義する。

$$
H = \sum_{v \in V, s \in S, (i,j) \in L} d_{ij} * x_{vsi} * x_{v(s+1)j}
+w_0\sum_{v \in V} (x_{v00} - 1)^2
+w_1\sum_{v \in V} (x_{v(S-1)0} - 1)^2
+w_2\sum_{v \in V, s \in S}\left(\sum_{l \in L} x_{vsl} - 1\right)^2
+w_3\sum_{l \in L, l!=0}\left(\sum_{v \in V, s \in S} x_{vsl} - 1\right)^2
$$

In [11]:
# Vehicle Routing
x = pyqubo.Array.create('x', shape=(V,S,L), vartype='BINARY')

In [12]:
%%time
H = build_objective(x) + \
  pyqubo.Placeholder('w_0') * build_depot_start_rule(x) + \
  pyqubo.Placeholder('w_1') * build_depot_end_rule(x) + \
  pyqubo.Placeholder('w_2') * build_vehicle_stay_rule(x) + \
  pyqubo.Placeholder('w_3') * build_location_visit_rule(x)

CPU times: user 1min 2s, sys: 32.3 ms, total: 1min 2s
Wall time: 1min 2s


### Hamiltonian Penalty Factor

* ハミルトニアンにおけるペナルティ係数 w0, w1, w2, w3 の値を大きくすることで、制約違反の発生確率を下げることができる。

* 一方、ペナルティ係数が極端に大きいことで、ハミルトニアンの収束が不安定となり、総移動コストが低い解を得るまでに時間を要する。

* そのため、ペナルティ係数は制約違反が起きない程度に小さくするチューニングが必要となる。

In [13]:
feed_dict = {'w_0': 1000,
             'w_1': 1000,
             'w_2': 1000,
             'w_3': 1000}

### Create QUBO

* 作成したハミルトニアンより、QUBO 式を構築する。

In [14]:
model = H.compile()
qubo, constant = model.to_qubo(feed_dict=feed_dict)

## Execute Quantum Annealing

* Sampler と呼ばれるモジュールを生成し、構築した QUBO 式を渡すことで、量子アニーリングを実行する。

* 結果は、Response オブジェクトとして返却される。

### OpenJij

* OpenJij（[GitHub](https://github.com/OpenJij/OpenJij), [Documentation](https://openjij.github.io/OpenJij_Documentation/build/html/), [Tutorial](https://openjij.github.io/OpenJijTutorial/build/html/ja/index.html)）

* OSS として、量子アニーリングをシミュレートする SQA（Simulated Quantum Annealing）の Python 用 API を提供している。

In [15]:
sampler = SQASampler(trotter=20, num_reads=10)

In [16]:
%%time
response = sampler.sample_qubo(qubo)

CPU times: user 1min 7s, sys: 1.53 s, total: 1min 8s
Wall time: 1min 6s


### D-Wave

* D-Wave Ocean SDK（[GitHub](https://github.com/dwavesystems/dwave-ocean-sdk), [Documentation](https://docs.ocean.dwavesys.com/en/latest/)）

* Web API を介して、量子アニーリングマシンを利用する。Python 用 SDK が提供されている。

* 利用にはアカウントが必要（無料利用枠あり）。発行される token と endpoint を指定する。

* LeapHybridSampler は、QPU（Quantum Processing Unit）と CPU の両方を用いて量子アニーリングを行う。多くの変数（Qubit）を利用する、大規模な問題にも対応できるように設計されている。

In [ ]:
# Upload the file with the token and endpoint
confidential = colab.files.upload()

In [ ]:
config = ConfigParser()
config.read([fn for fn in confidential.keys()][0])

In [19]:
sampler = LeapHybridSampler(token=config['API']['token'], endpoint=config['API']['endpoint'])

In [20]:
%%time
response = sampler.sample_qubo(qubo)

CPU times: user 246 ms, sys: 23 ms, total: 269 ms
Wall time: 3.13 s


### Take Solutions

* Response オブジェクトに格納されている量子アニーリングの結果（解）を取り出す。

In [21]:
def extract_samples(response) -> (list, list):
  solutions = []
  energies = []
  
  for record in response.record:
    sol, num_occ = record[0], record[2]
    solution, broken, energy = model.decode_solution(dict(zip(response.variables, sol)), vartype='BINARY', feed_dict=feed_dict)
    if len(broken) == 0:
      solutions += [solution] * num_occ
      energies += [energy] * num_occ
  
  return solutions, energies

In [22]:
solutions, energies = extract_samples(response)

### Select Best Solution

* 総移動コストが最小の解を選び、pandas の DataFrame に変換する。

In [23]:
def vehicle_movement(solution: list) -> list:
  vehicles = []

  for v in range(V):
    solution_sorted = sorted(solution['x'][v].items(), key=lambda x:x[0])
    s = [i[1] for i in solution_sorted]
    df = pd.DataFrame(s).astype(int)
    df.columns = [chr(l) for l in range(65, 65+L)]
    vehicles.append(df)
  
  return vehicles

In [24]:
best_solution = solutions[energies.index(min(energies))]
best_vehicles = vehicle_movement(best_solution)

### Print Vehicle Routing

* 各配送車が、各訪問順番において、どの地点に滞在するのかを表示する。

* 先に設定した制約条件が満たされていることを確認してください。

> 最初と最後の訪問では必ず Depot（A） に滞在する。
>
> 全ての配送車及び全ての訪問順番で、必ず一つの拠点に訪問する。
>
> いずれかの配送車及びいずれかの訪問順番で、Depot（A） 以外の全ての拠点が必ず一回ずつ訪問される。

In [25]:
def highlight_positive(val: int) -> str:
  if val == 1:
    return 'color: {}; font-weight: bold'.format('red')
  else:
    return 'color: {}'.format('gray')

for v in range(V):
  display_html('<b>vehicle-{}</b>'.format(v+1) + best_vehicles[v].style.applymap(highlight_positive)._repr_html_() + '<br>', raw=True)

,A,B,C,D,E,F,G,H,I,J,K,L,M,N,O,P,Q,R,S,T,U,V
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
6,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


,A,B,C,D,E,F,G,H,I,J,K,L,M,N,O,P,Q,R,S,T,U,V
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
8,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


,A,B,C,D,E,F,G,H,I,J,K,L,M,N,O,P,Q,R,S,T,U,V
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
9,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


## Evaluation

* 量子アニーリングは、「解が出たから終わり」ではなく、「解が出てからが始まり」である。

* パラメータチューニングを繰り返して、よりよい解が見つかるように調整する必要がある。

### Energy

* 目的関数の値（本問題では総移動コスト）を、量子アニーリングでは特に「エネルギー」と呼ぶ。

* 「解の質」そのものであり、手法間比較や、パラメータチューニングにおいて、最も重要な値である。

In [26]:
# Energy of Best Solution
'''
Ex: OpenJij -> 7598.0
'''
min(energies)

6222.0

## Conclusion

* 配送計画問題を題材に、量子アニーリングの威力を体験した。

* D-Wave の LeapHybridSampler による量子アニーリングは、2020年9月現在、10,000 変数（Qubit）まで使用することができる。

* ハード・ソフト共に、更なる発展が楽しみな分野である。